# Start Here Notebook


This notebook is used to make sure that your setup is ready to run for the workshop.

The variables that will be created are

* `project_prefix`: Project name prefix to be used for resource naming, e.g. S3, training job, endpoint name, pipeline names, etc.
* `bucket_prefix`:  The main bucket that will be used throughout the examples.
* `mlflow_name`: The name of the Mlflow server.
* `mlflow_arn`: The resource identifier for the Mlflow server to be used for tracking experiments and runs.
* `domain_id`: The Sagemaker domain id.
* `region`: The current AWS region being used.



To run this notebook and all notebooks in the workshop please use the `Python 3` kernel in JupyterLab

## Setup
Get the latest version of SageMaker Python SDK.

<div class="alert alert-info"> 💡 The workshop and all notebooks were tested with Sagemaker Distribution `1.11` and the SageMaker Python SDK (the package sagemaker) version 2.219.0. The notebooks don't pin the version of the sagemaker. If you encounter any incompatibility issues, you can install the specific version of the sagemaker by running the pip command: <code>%pip install sagemaker=2.219.0</code>
</div>

### Import packages

In [ ]:
import os
import boto3
import sagemaker
import json

### Set constants

In [ ]:
# Get some variables you need to interact with SageMaker service
boto_session = boto3.Session()
sm_session = sagemaker.Session()
sm_client = boto_session.client("sagemaker")
bucket_name = sm_session.default_bucket()

region = boto_session.region_name
project_prefix = "amzn"
bucket_prefix = f"{bucket_name}/{project_prefix}"

initialized = True

In [ ]:
# Store some variables to keep the value between the notebooks

%store project_prefix
%store bucket_prefix
%store region
%store initialized

### Get domain id
You need this value `domain_id` in many SageMaker Python SDK and boto3 SageMaker API calls. The notebook metadata file contains `domain_id` value. The following code demonstrates how to access the notebook metadata file and get the `domain_id`.

In [ ]:
NOTEBOOK_METADATA_FILE = "/opt/ml/metadata/resource-metadata.json"
domain_id = None

if os.path.exists(NOTEBOOK_METADATA_FILE):
    with open(NOTEBOOK_METADATA_FILE, "rb") as f:
        metadata = json.loads(f.read())
        domain_id = metadata.get('DomainId')        

assert domain_id
%store domain_id

### Get MLflow server
If you're running an AWS-led workshop or used the delivered CloudFormation template to provision your workshop environment, an MLflow server must be up and running. If you don't have an MLflow server, follow the [Developer Guide](https://docs.aws.amazon.com/sagemaker/latest/dg/mlflow-create-tracking-server.html) to create one. The creation of a new MLflow server can take up to 25 minutes.

Execute the following code to get the name and ARN of the MLflow server.

In [ ]:
def get_mlflow_server_arn():
    list_servers = boto3.client("sagemaker").list_mlflow_tracking_servers()['TrackingServerSummaries']

    assert len(list_servers)==1 # should be one MLflow server, if not make sure to set your correct one        
    mlflow_arn = list_servers[0]['TrackingServerArn']
    mlflow_name = list_servers[0]['TrackingServerName']
    print(f"MLflow Server Name: {mlflow_name}, ARN: {mlflow_arn}")    
    return mlflow_name,mlflow_arn

mlflow_name, mlflow_arn = get_mlflow_server_arn()

In [ ]:
%store mlflow_name
%store mlflow_arn

### Check if docker access is enabled

In [ ]:
# check that docker enabled in the SageMaker domain
docker_settings = sm_client.describe_domain(DomainId=domain_id)['DomainSettings'].get('DockerSettings')
docker_enabled = False

if docker_settings:
    if docker_settings.get('EnableDockerAccess') in ['ENABLED']:
        print(f"The docker access is ENABLED in the domain {domain_id}")
        docker_enabled = True

if not docker_enabled:
    raise Exception(f"You must enable docker access in the domain to use Studio local mode")

<div style="border: 4px solid coral; text-align: center; margin: auto;">
If the previous code cell raised an exeption that the docker access is not enabled, you need to enable the access. <br>See the <a href="01-custom-environment-guidance.ipynb">01-custom-environment-guidance.ipynb</a> for instructions how to do it.
</div>

## Further workshop flow
Continue with the workshop by going to Module 01.
